In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +datetime.timedelta(hours=t.minute//30))


In [3]:
df_nanumaga = pd.read_csv('Nanumaga_dates.csv')
df_nanumea = pd.read_csv('Nanumea_dates.csv')

df_nanumea = df_nanumea[~df_nanumea.time.isna()]
df_nanumaga = df_nanumaga[~df_nanumaga.time.isna()]

In [4]:
df_funafuti_tides = pd.read_csv('../Additional_data/Tidal_data/Data/h025.csv')

In [5]:
df_funafuti_tides.index = [datetime.datetime(y,m,d,h) for y,m,d,h in zip(df_funafuti_tides.year,df_funafuti_tides.month,df_funafuti_tides.date,df_funafuti_tides.hour)]


In [6]:
nanumea_datetimes = [datetime.datetime(int(d.split('/')[2]),int(d.split('/')[1]),int(d.split('/')[0]),
                   int(t.split(':')[0]),int(t.split(':')[-1].split('a')[0]))
    for d,t in zip(df_nanumea.date,df_nanumea.time)
]
df_nanumea.index = [hour_rounder(x) for x in nanumea_datetimes]
df_nanumea.drop('date',axis=1,inplace=True)

# Add the tide level to the dataframe
df_nanumea = df_nanumea.join(df_funafuti_tides)
df_nanumea = df_nanumea[df_nanumea.level>0] # remove missing values

nanumaga_datetimes = [datetime.datetime(int(d.split('/')[2]),int(d.split('/')[1]),int(d.split('/')[0]),
                   int(t.split(':')[0]),int(t.split(':')[-1].split('a')[0]))
    for d,t in zip(df_nanumaga.date,df_nanumaga.time)
]
df_nanumaga.index = [hour_rounder(x) for x in nanumaga_datetimes]
df_nanumaga.drop('date',axis=1,inplace=True)

# Add the tide level to the dataframe
df_nanumaga = df_nanumaga.join(df_funafuti_tides)
df_nanumaga = df_nanumaga[df_nanumaga.level>0] # remove missing values


In [7]:
# Calculate gradient
nanumea_dict = {}
for index,row in df_nanumea.iterrows():
    look_back_grad = df_funafuti_tides.loc[index,'level']-df_funafuti_tides.loc[(index-datetime.timedelta(hours=1)),'level']
    look_forward_grad = df_funafuti_tides.loc[(index+datetime.timedelta(hours=1)),'level']-df_funafuti_tides.loc[index,'level']
    row['grad'] = np.mean([look_back_grad,look_forward_grad])
    
    nanumea_dict.update({index:row})
    
df_nanumea = pd.DataFrame.from_dict(nanumea_dict,orient='index')

In [8]:
# Calculate gradient
nanumaga_dict = {}
for index,row in df_nanumaga.iterrows():
    look_back_grad = df_funafuti_tides.loc[index,'level']-df_funafuti_tides.loc[(index-datetime.timedelta(hours=1)),'level']
    look_forward_grad = df_funafuti_tides.loc[(index+datetime.timedelta(hours=1)),'level']-df_funafuti_tides.loc[index,'level']
    row['grad'] = np.mean([look_back_grad,look_forward_grad])
    
    nanumaga_dict.update({index:row})
    
df_nanumaga = pd.DataFrame.from_dict(nanumaga_dict,orient='index')

In [9]:
df_nanumaga.to_csv('nanumaga_time_with_tide.csv',index=True)

In [10]:
df_nanumea.to_csv('nanumea_time_with_tide.csv',index=True)